# HDS5210-2022 Midterm

In the midterm, you're going to use all of the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged: "home" if they are below 30% risk and the recommended hospital if they are at or above 30%.

Each step in the midterm will build up to form your final solution. Along the way, I've provided plenty of test cases to make sure that you're getting those steps correct.

The midterm is due at 11:59 PM CST on Monday, March 14th.

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should raise a ValueException that includes a message with the invalid input and which parameter that input was provided as.

NOTES:
1. In the final step there is a rule to convert form raw score to percentile.  In that conversion, 17-25 maps to 59-88% and ≥26 maps to >99%.  For our purposes, we want these to be specific % number outputs.  Use the following rule:
 * If score is between 17 and 25, percentile should be 0.59
 * If score is greater than or equal to 26, percentile should be 0.99


In [1]:
import requests
import json
import html5lib
import os
import pandas as pd
from bs4 import BeautifulSoup as BS

In [2]:
def get_sex_score(sex):
    if sex == "male": return 1
    if sex == "female": return 0
    raise ValueError('parameter sex is invalid input')
    return 0   

In [3]:
def get_age_score(age):
  if 16 <= age <= 49: return 0
  elif 50 <= age <= 65: return 2
  elif 66 <= age <= 80: return 3
  elif age > 80: return 4
  return 0

In [4]:
def get_rrbp_score(rrbp):
  if rrbp < 9: return 3
  elif 9 <= rrbp <= 11: return 1
  elif 12 <= rrbp <= 20: return 0
  elif 21 <= rrbp <= 24: return 2
  elif rrbp >24: return 3
  return 0

In [5]:
def get_os_score(os):
  if os > 95: return 0
  elif 94 <= os <= 95: return 1
  elif 92 <= os <= 93: return 2
  elif os < 92: return 3
  return 0

In [6]:
def get_hrbp_score(hrbp):
  if hrbp < 41: return 3
  elif 41 <= hrbp <= 50: return 1
  elif 51 <= hrbp <= 90: return 0
  elif 91 <= hrbp <= 110: return 1
  elif 111 <= hrbp <= 130: return 2
  elif hrbp > 130: return 3
  return 0

In [7]:
def get_sbp_score(sbp):
    if sbp < 91: return 3
    elif 91 <= sbp <= 100: return 2
    elif 101 <= sbp <= 110: return 1
    elif 111 <= sbp <= 219: return 0
    elif sbp > 219: return 3
    return 0

In [8]:
def get_temp_score(temp):
    if temp < 35.1: return 3
    elif 35.1 <= temp <= 36: return 1
    elif 36.1 <= temp <= 38: return 0
    elif 38.1 <= temp <= 39.0: return 1
    elif temp > 39: return 2
    return 0

In [9]:
def get_misc_score(misc):
  store = {"alert": 0, "confused or not alert" : 3, "air": 0, "supplemental oxygen" : 2, "unrestricted normal activity": 0, "limited strenuous activity, can do light activity": 1, "limited activity, can self-care" : 2, "limited self-care" : 3, "bed/chair bound, no self-care" : 4}
  if misc not in store: raise ValueError(misc, " is invalid input")
  return store[misc]

In [10]:
def priest(sex, age, rrate, spo2, hrate, bp, temp, alert, inspired_oxygen, performance):
    """
    The priest function computes(gets the score from each of the input argument) and returns the risk percentage by considering input factors.

    Parameters:
      sex (stirng) : gender of the input person
      age (integer) : age of the person in years
      rrate (float) : Respiratory rate in breaths per minute
      spo2 (float) : Oxygen saturation as a percent between 0 and 1
      hrate (float) : Heart rate in beats per minute
      bp (float) : Systolic BP in mmHg
      temp (float) : Temperature in degrees C
      alert (string) : Alertness as a string description
      inspired_oxygen (string) : Inspired Oxygen as as string description
      performance (string) : Performance Status as a string description

    Returns:
      Float: Returns the risk percentage after calculating from the input args and after the mapping with the severity
      Raises ValueError with the invalid argument if invalid input is entered  

    >>> priest("FEMALE", 40, 24, 96, 105, 115, 34.9, "ALERT", "AIR", "unrestricted normal activity")
    0.15
    >>> priest("Female", 40, 22, 98, 89, 230, 38.1, "alert", "AIR", "unrestricted normal activity")
    0.15
    >>> priest("FEMALE", 18, 23, 96, 105, 101, 38.1, "ALERT", "AIR", "limited activity, can self-care")
    0.18
    >>> priest("male", 65, 23, 99, 105, 91, 38.1, "alert", "AIR", "unrestricted normal activity")
    0.26
    """
    score = 0
    score += get_sex_score(sex.lower()) 
    score += get_age_score(age)
    score += get_hrbp_score(hrate)
    score += get_os_score(spo2*100)
    score += get_rrbp_score(rrate)
    score += get_sbp_score(bp)
    score += get_temp_score(temp)
    score += get_misc_score(alert.lower())
    score += get_misc_score(inspired_oxygen.lower())
    score += get_misc_score(performance.lower())

    conversion_dict = {0: 1,
                        1: 1,
                        2: 2,
                        3: 2,
                        4: 3,
                        5: 9,
                        6: 15,
                        7: 18,
                        8: 22,
                        9: 26,
                        10: 29,
                        11: 34,
                        12: 38,
                        13: 46,
                        14: 47,
                        15: 49,
                        16: 55}
    
    if score<=16: return conversion_dict[score] / 100
    elif score>=17 and score<=25: return 0.59
    else: return 0.99

import doctest
doctest.testmod()


TestResults(failed=0, attempted=4)

## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk percetage of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None without raising any exceptions
5. Include a good docstring with at least five test cases.


In [11]:
def find_hospital(age, sex, risk):
  """
  The find_hospital function takes the age, sex and risk and gets the hospital name by triggering the "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/" with the given args and then returns the name of the hospital after parsing the json. 
  
  Parameters: 
    sex (stirng) : gender of the input person
    age (integer) : age of the person in years
    risk(float) : risk returned from the priest function.

  Returns:
    The hospital name matching with given parameters after triggering the URL(which has the whole logic)

  >>> find_hospital(40, "male", 0.1)
  'Southwest Hospital and Medical Center'
  >>> find_hospital(40, "female", 0.26)
  'Select Specialty Hospital - Northeast Atlanta'
  >>> find_hospital(30, "male", 0.18)
  'Southwest Hospital and Medical Center'
  >>> find_hospital(22, "female", 0.59)
  'Emory Dunwoody Medical Center'
  >>> find_hospital(65, "male", 0.99)
  'Wesley Woods Geriatric Hospital'
  """
  get_details_url="https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=" + str(age) + "&sex=" + sex + "&risk_pct=" + str(risk)
  request_get_details = requests.get(get_details_url).json() 
  if not request_get_details: return None
  else: hospital = request_get_details['hospital'] 
  return hospital

import doctest
doctest.testmod()

TestResults(failed=0, attempted=9)

## Part 3: Get the address for that hospital from a webpage

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list on this webpage to lookup the address for that hospital, based on its name.

https://www.officialusa.com/stateguides/health/hospitals/georgia.html

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data on the webpage above to find the addres for that hospital.
2. I've said that all the hospitals are in Atlanta, but this webpage has hospitals from all of Georgia.  So, make sure you verify that the row of data you're using is in Atlanta, just in case there are hospitals with the same name in different cities.
3. Your code will have to parse the HTML on the webpage and turn that into some kind of data structure that you can search through to find the right hospital.
4. If you do find more than one matching hospital in Atlanta with the same name, you should raise a KeyError.
5. If the hospital name isn't found, the function should raise a KeyError.
6. Be sure to use good docstring documentation and includes at least 3 test cases.

In [12]:
atlanta_hospitals_address = {}

def get_address(name):
    """
    The get_address function takes in the name of the hospital and returns the address of the hospital with utilization of the data at "https://www.officialusa.com/stateguides/health/hospitals/georgia.html"

    Parameters:
      name (string): name of the hospital
    
    Returns:
      The address of the hospital given

    >>> get_address("CHARTER BEHAVIORAL HEALTH SYSTEM OF ATLANTA")
    '811 JUNIPER STREET NE'
    >>> get_address("SELECT SPECIALTY HOSPITAL - ATLANTA")
    '550 PEACHTREE STREET, NE'
    >>> get_address("SELECT SPECIALTY HOSPITAL - MIDTOWN ATLANTA")
    '705 JUNIPER STREET NE'
    """
    name = name.upper()
    if name in atlanta_hospitals_address: return atlanta_hospitals_address[name]
    response = requests.get("https://www.officialusa.com/stateguides/health/hospitals/georgia.html")
    res_content = BS(response.content,'html5lib')

    georgia_table = res_content.find('table',attrs={'id':'myTable'})
    all_rows = georgia_table.findChildren('tr')[1:]

    for row in all_rows:
        children = row.findChildren('td')
        state = [child.string for child in children]
        if state[0] == 'ATLANTA': atlanta_hospitals_address[state[1]] = state[3]
            
    return atlanta_hospitals_address[name]

import doctest
doctest.testmod()

TestResults(failed=0, attempted=12)

## Part 4: Run the risk calculator on a population

The `/data` folder has a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.

In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes a file name as a parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you below.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [13]:
import pandas as pd
def process_people(file_name):
    #path=os.path.join("./data",file_name)
    df = pd.read_csv(open(file_name,'r'),sep="|")
    all_info = dict()
    for i in range(len(df)):
        row = df.iloc[i]
        calc_risk = priest(row.sex, row.age, row.breath, row['o2 sat'], row.heart, row['systolic bp'], row.temp, row.alertness, row.inspired, row.status)
        hospital = find_hospital(row.age, row.sex, calc_risk)
        address = get_address(hospital)
        all_info[row.patient]=[row.sex, row.age, row.breath, row['o2 sat'], row.heart, row['systolic bp'], row.temp, row.alertness, row.inspired, row.status, calc_risk, hospital, address]
    return all_info

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` in the `/data` folder.  Write some code to check your results.  This does not need to be a function.

In [14]:
expected_outputs=json.load(open("../data/people_results.json",'r'))
expected_outputs == process_people("../data/people.psv")

True

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

In order to submit your work, you'll need to use the `git` command line program to **add** your homework file (this file) to your local repository, **commit** your changes to your local repository, and then **push** those changes up to github.com.  From there, I'll be able to **pull** the changes down and do my grading.  I'll provide some feedback, **commit** and **push** my comments back to you.  Next week, I'll show you how to **pull** down my comments.

To run through everything one last time and submit your work:
1. Use the `Kernel` -> `Restart Kernel and Run All Cells` menu option to run everything from top to bottom and stop here.
2. Follow the instruction on the prompt below to either ssave and submit your work, or continue working.

If anything fails along the way with this submission part of the process, let me know.  I'll help you troubleshoort.

---

In [16]:
a=input('''
Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

''')

if a=='yes':
    !git add "midterm-2022.ipynb"
    !git commit -a -m "Submitting the midterm"
    !git push
else:
    print('''
    
OK. We can wait.
''')


Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

 yes


On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
	Untitled.ipynb
	Untitled1.ipynb
	apache_patients.csv
	apache_rules.json
	extra-2022.ipynb
	week 03/week03_class.ipynb
	week 04/week04_examples.ipynb
	week 04/week04_lookups.ipynb
	week 05/
	week 06/module29-xml-examples.ipynb
	week 06/week06_inclass_1.ipynb
	week 10/
	week02/

nothing added to commit but untracked files present
Everything up-to-date
